In [ ]:
import re
import json
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
document_names = [
                    'Определение Верховного Суда РФ от 10.12.2018 N 310-ЭС18-1678.htm',
                    'Определение Верховного Суда РФ от 10.12.2018 N 305-ЭС18-1282.htm',
                    'Определение Верховного Суда РФ от 10.12.2018 N 305-ЭС18-1564.htm',
                    'Определение Верховного Суда РФ от 10.12.2018 N 305-ЭС18-1703.htm',
                    'Определение Верховного Суда РФ от 10.12.2018 N 305-ЭС18-1726.htm',
                    'Определение Верховного Суда РФ от 10.12.2018 N 306-ЭС17-2034.htm',
                    'Определение Верховного Суда РФ от 10.12.2018 N 306-ЭС18-8105.htm',
                    'Определение Верховного Суда РФ от 10.12.2018 N 307-ЭС18-1126.htm',
                    'Определение Верховного Суда РФ от 10.12.2018 N 308-ЭС17-1033.htm',
                    'Определение Верховного Суда РФ от 13.12.2018 по делу N 305-Э.htm',
                    'Определение Верховного Суда РФ от 11.12.2018 N 305-ЭС18-1761.htm',
                    'Определение Верховного Суда РФ от 11.12.2018 N 5-КГ18-247 Тр.htm',
                    'Определение Верховного Суда РФ от 12.12.2018 N 305-ЭС18-1664.htm',
                    'Определение Верховного Суда РФ от 12.12.2018 N 305-ЭС18-1690.htm',
                    'Определение Верховного Суда РФ от 13.12.2018 N 301-ЭС18-1341.htm',
                    'Определение Верховного Суда РФ от 13.12.2018 N 305-ЭС18-1411.htm',
                    'Определение Верховного Суда РФ от 13.12.2018 N 305-АД18-1492.htm',
                    'Определение Верховного Суда РФ от 13.12.2018 N 305-ЭС18-5703.htm',
                    'Определение Верховного Суда РФ от 13.12.2018 по делу N 303-Э.htm',
                    'Определение Верховного Суда РФ от 13.12.2018 по делу N 308-Э.htm',
                ]

In [ ]:
df = pd.read_excel('./разметка.xlsx')
# df.to_excel('./разметка.xlsx')

In [ ]:
def get_text_from_file(document_name):
    soup = BeautifulSoup(open(document_name), "lxml")
    temp = soup.getText()[soup.getText().find('ВЕРХОВНЫЙ СУД РОССИЙСКОЙ ФЕДЕРАЦИИ'):]
    text = temp[:temp.find('Документ предоставлен КонсультантПлюс')]
    return text

In [ ]:
with open('codes_regexp_patterns.json', 'r', encoding='utf-8') as f:
    codes_regexp_patterns = json.load(f)
with open('na_levels_regexp_patterns.json', 'r', encoding='utf-8') as f:
    na_levels_regexp_patterns = json.load(f)

In [ ]:
def get_marked_text(text):
    patterns = {**na_levels_regexp_patterns, **codes_regexp_patterns}
    for na_level in patterns:
        text = re.sub(patterns[na_level], na_level, text)
    return text

In [ ]:
def get_na_strings(marked_text):
    other_simbols = ['-', '\s', ',', 'и', '\.', '\d']
    start_form = '(' + '|'.join(['(' + form + '[' + ''.join(other_simbols) + ']+)' for form in na_levels_regexp_patterns]) + ')+'
    end_form = '(' + '|'.join(['(' + form + ')' for form in codes_regexp_patterns]) + ')'
    na_string_pattern = re.compile(start_form + end_form)
    return [marked_text[na_string.start():na_string.end()] for na_string in na_string_pattern.finditer(marked_text)]

In [ ]:
def get_na_strings(text, get_pos=False):
    other_simbols = ['-', '\s', ',', 'и', '\.', '\d']
    start_form = '(' + '|'.join(['(' + form + '[' + ''.join(other_simbols) + ']+)' for form in na_levels_regexp_patterns.values()]) + ')+'
    end_form = '(' + '|'.join(['(' + form + ')' for form in codes_regexp_patterns.values()]) + ')'
    na_string_pattern = re.compile(start_form + end_form)
    if get_pos:
        return [(na_string.start(),na_string.end()) for na_string in na_string_pattern.finditer(text)]
    else:
        return [text[na_string.start():na_string.end()] for na_string in na_string_pattern.finditer(text)]

In [ ]:
def are_equal(res1, res2):
    res = True
    for element in res1:
        if element not in res2:
            res = False
    for element in res2:
        if element not in res1:
            res = False
    return res

In [ ]:
for i in range(len(document_names)):
    document_name = document_names[i]
    my_res = get_na_strings(get_text_from_file(document_name))
    validate_res = [na_string for na_string in list(df[df['document']==document_name]['entity'])]
    print(are_equal(my_res, validate_res))

In [ ]:
i=12
document_name = document_names[i]
my_res = get_na_strings(get_text_from_file(document_name), get_pos=False)
validate_res = [na_string for na_string in list(df[df['document']==document_name]['entity'])]
print(document_name)
print(my_res == validate_res)
print('len of my_res: {}'.format(len(my_res)))
print('len of validate_res: {}'.format(len(validate_res)))
my_res

In [ ]:
validate_res

In [ ]:
validate_res = [get_marked_text(na_string) for na_string in list(df[df['document']==document_name]['entity'])]

In [ ]:
get_text_from_file(document_name)[9080:9159]

In [ ]:
import re

In [ ]:
import json

codes_regexp_patterns = {
                            'CodeOfAdministrativeOffenses' : 'А(дминистративно-)?[пП](равового)?\s?[кК](одекса)?(\sР(оссийской)?\s?Ф(едерации)?)?',
                            'CivilCode'                    : 'Г(ражданского)?\s?[кК](одекса)?(\sР(оссийской)?\s?Ф(едерации)?)?',
                            'CodeOfCommercialProcedure'    : 'А(рбитражного )?[пП](роцессуального)?\s?[кК](одекса)?(\sР(оссийской)?\s?Ф(едерации)?)?',
                            'BudgetCode'                   : 'Б(юджетного)?\s?[кК](одекса)?(\sР(оссийской)?\s?Ф(едерации)?)?',
                            'AirCode'                      : 'В(оздушного)?\s?[кК](одекса)?(\sР(оссийской)?\s?Ф(едерации)?)?',
                            'CodeOfAdministrativeOffenses' : '[кК](одекса)?(\\sР(оссийской)?\\s?Ф(едерации)?)?\\s?[оО]б?\\s?[аА](дминистративных)?\\s?[пП](равонарушениях)?(\\sР(оссийской)?\\s?Ф(едерации)?)?',
                            'TaxCode'                      : '[нН](алогового)?\s?[кК](одекса)?(\sР(оссийской)?\s?Ф(едерации)?)?',
                            'CodeofCivilProcedure'         : 'Г(ражданского )?[пП](роцессуального)?\s?[кК](одекса)?(\sР(оссийской)?\s?Ф(едерации)?)?',
                            'LandCode'                     : 'З(емельного)?\s?[кК](одекса)?(\sР(оссийской)?\s?Ф(едерации)?)?'
                        }
na_levels_regexp_patterns = {
                        'Paragraph'   :    '(([аА]бз\.)|([аА]бзац[ыауе]?м?))',
                        'SubSection'  :    '(([пП]одп\.)|([пП]одпункт[ыуеао]?[мв]?))',
                        'Section'     :    '(([пП]\.)|([пП]ункт[ыуеао]?[хмв]?))',
                        'ArticlePart' :    '(([чЧ]\.)|([чЧ]аст[еьи][йю]?))',
                        'Article'     :    '(([сС]т\.)|([сС]тать?[еяию]м?[ий]?))',
                        'CodeChapter' :    '(([гГ]л\.)|([гГ]лав[аыо]й?))',
                             }

json.dump(codes_regexp_patterns, open('codes_regexp_patterns.json', 'w'), ensure_ascii=False)
json.dump(na_levels_regexp_patterns, open('na_levels_regexp_patterns.json', 'w'), ensure_ascii=False)

In [ ]:
code_pattern = '|'.join(['(' + code + ')' for code in codes_regexp_patterns.values()])
re.sub(code_pattern, 'CODE', 'частей 2 и 3 статьи 53 Гражданского процессуального кодекса Российской Федерации')

In [1]:
from LegalDocument import LegalDocument as LD

In [2]:
na_strings = ['частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях', 'частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях', 'частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях', 'частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях', 'частью 3 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях', 'статьи 26.11 Кодекса Российской Федерации об административных правонарушениях', 'частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях', 'частью 2 статьи 2.1 Кодекса Российской Федерации об административных правонарушениях', 'частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях', 'статьи 24.1 Кодекса Российской Федерации об административных правонарушениях', 'статьями 3.1, 3.5, 4.1 - 4.3 Кодекса Российской Федерации об административных правонарушениях', 'частью 1 статьи 4.5 Кодекса Российской Федерации об административных правонарушениях', 'статьями 30.6, 30.9, 30.16 Кодекса Российской Федерации об административных правонарушениях', 'пунктов 2 - 4 части 2 статьи 30.17 Кодекса Российской Федерации об административных правонарушениях', 'статьи 30.13 и 30.17 Кодекса Российской Федерации об административных правонарушениях', 'частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях']

In [3]:
for na_string in na_strings:
    print(na_string)
    ld = LD(na_string)
    ld.na_list

частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях
частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях
частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях
частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях
частью 3 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях
статьи 26.11 Кодекса Российской Федерации об административных правонарушениях
частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях
частью 2 статьи 2.1 Кодекса Российской Федерации об административных правонарушениях
частью 10 статьи 12.21.1 Кодекса Российской Федерации об административных правонарушениях
статьи 24.1 Кодекса Российской Федерации об административных правонарушениях
статьями 3.1, 3.5, 4.1 - 4.3 Кодекса Российской Федерации об административных правонарушениях
частью 1 статьи 4.5 Кодекса Российско

In [ ]:
ld = LD('пунктов 2 - 3 части 2 статьи 30.17 Кодекса Российской Федерации об административных правонарушениях')
ld.na_list

In [ ]:
i=10

In [ ]:
i+=1
print(i)
document_name = document_names[i]
lg = LG(get_text_from_file(document_name))
# lg.get_marked_na_string('ст. 387 Гражданского процессуального кодекса Российской Федерации')
lg.get_na_strings(get_text_from_file(document_name))

In [ ]:
lg.na_list

In [ ]:
with open('./словарь юридических терминов.txt') as f:
    terms = f.read().split('\n')
# terms

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
normalized_terms = [[morph.parse(word)[0].normal_form for word in term.split(' ')] for term in terms]
normalized_terms

In [ ]:
document_name = document_names[0]
# text = get_text_from_file(document_name)
# text = re.sub('\.+\s?', '.', text.replace('\n', '.'))
paragraphs = text.replace('\n\n', '\n').replace('руб.','руб').replace('мкр.','руб').replace('д.', 'д').split('\n')
# sentences = 
paragraphs

normalized_sentences = [[morph.parse(word)[0].normal_form for word in word_tokenize(sentence)] for paragraph in paragraphs if paragraph for sentence in sent_tokenize(paragraph)]
normalized_sentences

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
text += 'аутсортинг'
text

In [ ]:
def is_list_contains_list(list_1, list_2):
    for element in list_2:
        if element not in list_1:
            return False
    return True

In [ ]:
list1 = [1,2,3,4]
list2 = [2,4,5]
is_list_contains_list(['аутсортинг', 'fff'], ['аутсортинг'])

In [ ]:
set([is_list_contains_list(sentence, term) for term in normalized_terms for sentence in normalized_sentences])

In [ ]:
terms